# Interactive UIs with Gradio

**Module 04 | Notebook 2 of 4**

Create beautiful, shareable web interfaces for your ML models in minutes.

## Learning Objectives

By the end of this notebook, you will be able to:
1. Create Gradio interfaces for model inference
2. Customize UI components
3. Share your demos publicly
4. Deploy to Hugging Face Spaces

---

In [1]:
%%capture
!pip install transformers torch gradio

In [2]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import warnings
warnings.filterwarnings('ignore')

print(f"Gradio version: {gr.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Gradio version: 5.50.0
Using device: cuda


---

## Why Gradio?

### From Model to Demo in 3 Lines

```python
import gradio as gr

def predict(text):
    return model(text)

gr.Interface(fn=predict, inputs="text", outputs="text").launch()
```

### Gradio Features

| Feature | Description |
|---------|-------------|
| **Quick setup** | Create UIs in minutes |
| **Pre-built components** | Text, image, audio, video |
| **Public sharing** | Share via link instantly |
| **HF Spaces** | Free hosting on Hugging Face |
| **API generation** | Automatic REST API |

---

## Basic Sentiment Analysis Interface

In [3]:
# Load model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = pipeline("sentiment-analysis", model=model_name, device=0 if torch.cuda.is_available() else -1)

print(f"Model loaded: {model_name}")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


Model loaded: distilbert-base-uncased-finetuned-sst-2-english


In [7]:
# Simple prediction function
def predict_sentiment(text):
    if not text.strip():
        return "Please enter some text."
    
    result = classifier(text)[0]
    emoji = "😊" if result['label'] == 'POSITIVE' else "😠"
    return f"{emoji} {result['label']} ({result['score']:.1%})"

# Test
print(predict_sentiment("This is amazing!"))

😊 POSITIVE (100.0%)


In [10]:
# Create basic interface
demo_basic = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(label="Enter text", placeholder="Type your text here..."),
    outputs=gr.Textbox(label="Prediction"),
    title="Sentiment Analysis",
    description="Analyze the sentiment of your text using DistilBERT.",
    examples=[
        ["I absolutely love this product!"],
        ["This is the worst experience ever."],
        ["It's okay, nothing special."]
    ]
)

# Launch (set share=True to get a public link)
demo_basic.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://841d06748a11be986b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

## Enhanced Interface with Labels

In [12]:
# Enhanced prediction with confidence scores
def predict_with_scores(text):
    if not text.strip():
        return {"Error": 1.0}
    
    result = classifier(text)[0]
    
    # Return as dict for Label component
    if result['label'] == 'POSITIVE':
        return {
            "POSITIVE 😊": result['score'],
            "NEGATIVE 😠": 1 - result['score']
        }
    else:
        return {
            "POSITIVE 😊": 1 - result['score'],
            "NEGATIVE 😠": result['score']
        }

# Create enhanced interface
demo_enhanced = gr.Interface(
    fn=predict_with_scores,
    inputs=gr.Textbox(
        label="Enter text to analyze",
        placeholder="Type or paste your text here...",
        lines=3
    ),
    outputs=gr.Label(label="Sentiment", num_top_classes=2),
    title="Advanced Sentiment Analysis",
    description="See confidence scores for positive and negative sentiment.",
    examples=[
        ["The movie was great but the ending was disappointing."],
        ["Best purchase I've ever made! Highly recommended!"],
        ["It broke after one day. Complete waste of money."]
    ],
    theme="soft"
)

demo_enhanced.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15a3afe30074247628.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

## Text Summarization Interface

In [14]:
# Load summarization model
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=0 if torch.cuda.is_available() else -1
)

def summarize_text(text, max_length, min_length):
    if not text.strip() or len(text) < 50:
        return "Please enter at least 50 characters of text."
    
    result = summarizer(
        text,
        max_length=int(max_length),
        min_length=int(min_length),
        do_sample=False
    )
    
    return result[0]['summary_text']

# Create summarization interface
demo_summarize = gr.Interface(
    fn=summarize_text,
    inputs=[
        gr.Textbox(
            label="Input Text",
            placeholder="Paste a long article or text to summarize...",
            lines=10
        ),
        gr.Slider(50, 200, value=130, step=10, label="Max Summary Length"),
        gr.Slider(20, 80, value=30, step=10, label="Min Summary Length")
    ],
    outputs=gr.Textbox(label="Summary", lines=5),
    title="📝 Text Summarization",
    description="Summarize long texts using BART.",
    examples=[
        ["""The Amazon rainforest, often referred to as the planet's lungs, is a vast 
        tropical rainforest occupying the Amazon basin in South America. It covers 
        approximately 5.5 million square kilometers and spans across nine countries, 
        with Brazil containing the majority. The rainforest houses an estimated 
        10% of all species on Earth, including over 400 billion trees. However, 
        deforestation has become a major concern, with thousands of square miles 
        being cleared annually for agriculture, logging, and development.""", 100, 30]
    ]
)

demo_summarize.launch(share=True, inline=True)

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cf27f1d78261362a62.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

## Multi-Tab Interface with Blocks

In [15]:
# Create a multi-tab application using Blocks
with gr.Blocks(title="NLP Toolkit", theme="soft") as demo_blocks:
    gr.Markdown("# 🛠️ NLP Toolkit\nMultiple NLP tasks in one interface.")
    
    with gr.Tabs():
        # Tab 1: Sentiment Analysis
        with gr.TabItem("😊 Sentiment"):
            with gr.Row():
                with gr.Column():
                    sentiment_input = gr.Textbox(
                        label="Text",
                        placeholder="Enter text for sentiment analysis...",
                        lines=3
                    )
                    sentiment_btn = gr.Button("Analyze", variant="primary")
                with gr.Column():
                    sentiment_output = gr.Label(label="Result", num_top_classes=2)
            
            sentiment_btn.click(
                fn=predict_with_scores,
                inputs=sentiment_input,
                outputs=sentiment_output
            )
        
        # Tab 2: Summarization
        with gr.TabItem("📝 Summarize"):
            with gr.Row():
                with gr.Column():
                    sum_input = gr.Textbox(
                        label="Long Text",
                        placeholder="Paste text to summarize...",
                        lines=6
                    )
                    with gr.Row():
                        max_len = gr.Slider(50, 200, value=100, label="Max Length")
                        min_len = gr.Slider(20, 80, value=30, label="Min Length")
                    sum_btn = gr.Button("Summarize", variant="primary")
                with gr.Column():
                    sum_output = gr.Textbox(label="Summary", lines=4)
            
            sum_btn.click(
                fn=summarize_text,
                inputs=[sum_input, max_len, min_len],
                outputs=sum_output
            )
    
    gr.Markdown("---\n*Powered by Hugging Face Transformers*")

demo_blocks.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a3cb8deadc91ac9bcc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

## Deploy to Hugging Face Spaces

Create a complete app.py file for Spaces deployment:

In [16]:
# Complete app for HF Spaces
spaces_app = '''
import gradio as gr
from transformers import pipeline
import torch

# Load models
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("sentiment-analysis", device=device)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)

def analyze_sentiment(text):
    if not text.strip():
        return {"Error": 1.0}
    result = classifier(text)[0]
    return {
        f"{result['label']} {'😊' if result['label'] == 'POSITIVE' else '😠'}": result['score'],
        "Other": 1 - result['score']
    }

def summarize(text, max_len, min_len):
    if len(text) < 50:
        return "Please enter at least 50 characters."
    result = summarizer(text, max_length=max_len, min_length=min_len)
    return result[0]['summary_text']

with gr.Blocks(title="NLP Toolkit") as app:
    gr.Markdown("# 🛠️ NLP Toolkit")
    
    with gr.Tabs():
        with gr.TabItem("Sentiment"):
            gr.Interface(
                fn=analyze_sentiment,
                inputs=gr.Textbox(lines=3),
                outputs=gr.Label(),
                examples=[["I love this!"], ["This is terrible."]]
            )
        
        with gr.TabItem("Summarize"):
            gr.Interface(
                fn=summarize,
                inputs=[
                    gr.Textbox(lines=6),
                    gr.Slider(50, 200, value=100),
                    gr.Slider(20, 80, value=30)
                ],
                outputs=gr.Textbox()
            )

app.launch()
'''

# Save for deployment
with open("./gradio_app.py", "w") as f:
    f.write(spaces_app)

print("✅ App saved to gradio_app.py")
print("\nTo deploy to Hugging Face Spaces:")
print("1. Create a new Space at huggingface.co/new-space")
print("2. Select 'Gradio' as the SDK")
print("3. Upload gradio_app.py as 'app.py'")
print("4. Add requirements.txt with: transformers, torch, gradio")

✅ App saved to gradio_app.py

To deploy to Hugging Face Spaces:
1. Create a new Space at huggingface.co/new-space
2. Select 'Gradio' as the SDK
3. Upload gradio_app.py as 'app.py'
4. Add requirements.txt with: transformers, torch, gradio


---

## 🎯 Student Challenge

### Challenge: Create a Custom Demo

In [ ]:
# TODO: Create a Gradio demo with:

# 1. Text Generation tab (use "gpt2" pipeline)
# 2. Named Entity Recognition tab (use "ner" pipeline)
# 3. Add custom CSS styling
# 4. Include example inputs

# Hint:
# generator = pipeline("text-generation", model="gpt2")
# ner = pipeline("ner", grouped_entities=True)

# Your solution:


---

## 📝 Key Takeaways

1. **Gradio simplifies** creating ML demos
2. **gr.Interface** is quick, **gr.Blocks** offers more control
3. **Public links** let you share demos instantly
4. **Hugging Face Spaces** provides free hosting
5. **Automatic API** is generated for programmatic access

---

## ➡️ Next Steps

Continue to `03_docker_packaging.md` for containerization!